# Homework exercise 2
## Deadline: upload to Moodle by 31 May 18:00 h

__Please submit your homework either as a Jupyter Notebook or using .py files.__

If you use .py files, please also include a PDF containing the output of your code and your explanations. Either way, the code needs to be in a form that can be easily run on another computer.

__Name 1:__Julia Ornatowski

__Name 2:__Fabian Basler

__Name 3:__Evamaria Hammerschmid


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 1 June to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Selenium


__Yahoo Finance News Crawling__

Your task is to collect and organize articles from Yahoo Finance News, available at https://finance.yahoo.com/news

This will require you to use Selenium for at least two reasons:

* The site initially loads only partially such that scrolling is necessary to access additional news articles
* The news articles themselves, if they are long, are initially shown (and downloadable) in an abbreviated form. Browser navigation is necessary to click on a button such that the whole article is displayed.

There are about 200 news articles available at a time, stemming from various sources (e.g. Bloomberg, Reuters), and covering different topic areas (e.g. Business, World).

Note that you will need to use some features of Selenium - in particular scrolling - that are not discussed in the class notebook but documented elsewhere. 

You are asked to

* Download all of the about 200 news articles available from Yahoo Finance News
* Exclude the advertisements listed in between the news articles
* Try and remove everything that is not part of the article itself (e.g. the sidebar containing a list of popular articles and other content) and save the text of each article in a text file
* Create a DataFrame containing the following information (to the extent it is available) for each article
    * Title
    * Source (e.g. Bloomberg)
    * Time and date of publication
    * Name and path of the file containing the article's text that you saved
    
Please save this DataFrame so that you can later reuse the data set you created.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import json
from selenium import webdriver
import re
import random
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
#option to not see the browser opening
options = Options()
options.headless = True  

main_url = "https://finance.yahoo.com/news"
driver = webdriver.Chrome(options=options)  

driver.get(main_url)
sleep_sec= 4

In [3]:
# accepts the terms field
driver.find_element_by_xpath("//button[@type='submit']").click()
# loop as long there get new posts loaded on the site. all posts are inside the "li" tag
init_li=0
new_li = 1
while new_li > init_li:
    init_li = len(driver.find_elements_by_tag_name("li"))
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(sleep_sec)
    new_li=len(driver.find_elements_by_tag_name("li"))

In [4]:
time.sleep(sleep_sec)
# gets all urls of posts including ads
posts_url = [x.get_attribute("href") for x in driver.find_elements_by_xpath("//h3/a")]
# gets only the urls which lead to a yahoo finance news site
articles_url = [x for x in posts_url if "https://finance.yahoo.com/news" in x]

In [5]:
print(len(posts_url),len(articles_url))

192 150


In [6]:
#creating folder, path
dir_save="yahoo_finance_news" 
base_path = os.getcwd()
articles_paths = os.path.join(base_path, dir_save)
dir_saved= os.path.join(articles_paths, dir_save)
try:
   os.mkdir(dir_save)
   print("directory created")
except:
   print("directory exists")

directory created


In [7]:
source=[]
title=[]
time=[]
text=[]
paths=[]
txt_names = []

for counter, article in enumerate(articles_url,1):
    driver.get(article)
#agreing to terms and conditions of webpage
    try:
        button_agree = driver.find_element_by_xpath("//button[@type='submit']").click()
        button_agree.click()
    except:
        pass
    # clicks button to see whole text
    try:
        driver.find_element_by_xpath("//button[contains( text(), 'Story continues')]").click()
    except:
        pass
    #title
    title_s = driver.find_element_by_tag_name('h1')
    #time
    time_s= driver.find_element_by_tag_name('time')
    #text content
    text_s=driver.find_element_by_class_name('caas-body')
    #source
    sources=driver.find_element_by_css_selector('header>div>a>img') 
    source_s=sources.get_attribute('alt')
    source.append(source_s) 
    title.append(title_s.text) 
    time.append(time_s.text) 
    # text.append(text_s.text)
    # specifies name and path if the html
    txt_name = f'{counter}_article.txt'
    txt_path = os.path.join(articles_paths, txt_name)
    # crates text of the whole article
    article_content = f"{title_s.text}\n{text_s.text}"
    paths.append(f'\{dir_save}\{txt_name}')
    txt_names.append(txt_name)
    
    with open(txt_path, 'w') as f:
        f.write(article_content)
article_infos = pd.DataFrame({'Title':title,'Source':source,'Datetime': time, 'file':txt_names, 'path':paths})
driver.close()

In [8]:
article_infos

,Title,Source,Datetime,file,path
0,Vegan Activists Block McDonald’s U.K. Burger S...,Bloomberg,"Sat, May 22, 2021, 1:03 PM",1_article.txt,\yahoo_finance_news\1_article.txt
1,ViacomCBS streaming head: We're 'extremely wel...,Yahoo Finance,"Sat, May 22, 2021, 1:00 PM",2_article.txt,\yahoo_finance_news\2_article.txt
2,UPDATE 1-Campaign group Animal Rebellion block...,Reuters,"Sat, May 22, 2021, 12:48 PM",3_article.txt,\yahoo_finance_news\3_article.txt
3,U.S. Weighs Financial Sanctions Against Ethiop...,Bloomberg,"Sat, May 22, 2021, 12:18 PM",4_article.txt,\yahoo_finance_news\4_article.txt
4,German cyber security chief fears hackers coul...,Reuters,"Sat, May 22, 2021, 12:07 PM",5_article.txt,\yahoo_finance_news\5_article.txt
...,...,...,...,...,...
145,"How Shake Shack, McDonald's and others are len...",Yahoo Finance,"Fri, May 21, 2021, 6:18 PM",146_article.txt,\yahoo_finance_news\146_article.txt
146,Warren Buffett gave this investing tip to NFL ...,Yahoo Finance,"Fri, May 21, 2021, 5:16 PM",147_article.txt,\yahoo_finance_news\147_article.txt
147,New Stimulus Checks Could Start Hitting Your A...,GOBankingRates,"Fri, May 21, 2021, 12:20 PM",148_article.txt,\yahoo_finance_news\148_article.txt
148,Can employers legally require unvaccinated wor...,Yahoo Finance,"Thu, May 20, 2021, 2:13 PM",149_article.txt,\yahoo_finance_news\149_article.txt


In [9]:
#saves df
csv_path= os.path.join(articles_paths, "articles.csv")
article_infos.to_csv(csv_path)